In [ ]:
from os import listdir
from os.path import isfile, join
import os
from PIL import Image
import PIL
from tqdm.notebook import tqdm
import shutil 

!mkdir flattendata
!mkdir dataset

# make dataset - tarball-lite
!mkdir ./data 
!git clone https://github.com/afad-dataset/tarball-lite.git
%cd /content/tarball-lite
!cat AFAD-Lite.tar.xz* > AFAD-Lite.tar.xz
!tar -xvf AFAD-Lite.tar.xz -C /content/data
%cd -

# make dataset All-Age-Faces
!wget -O Dataset.zip https://www.dropbox.com/s/a0lj1ddd54ns8qy/All-Age-Faces Dataset.zip?dl=0
!unzip -qq 'Dataset.zip' -d /content/data

!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git
#!pip install ninja==1.10.2.2

orgpath = './data/All-Age-Faces Dataset/original images'
tarpath = './flattendata'
for path in os.listdir(orgpath):
    age = int(path.split('.')[0][-2:])
    if  (age > 40) or (5 < age < 18) :
        src = os.path.join(orgpath, path)
        image = PIL.Image.open(src)
        resized_image = image.resize((256,256))
        resized_image.save(os.path.join(tarpath, path))

def flatten_data(destination):
    paths = []
    for p in os.walk('/content/data/AFAD-Lite'):
        if len(p[2]) != 0:
            if 'jpg' in p[2][0]:
                for path in p[2]:
                    image_path = os.path.join(p[0], path)
                    paths.append(image_path)

    for image_path in tqdm(paths):
        image = PIL.Image.open(image_path)
        resized_image = image.resize((256,256))
        resized_image.save(os.path.join(destination, image_path.split('/')[-1]))

flatten_data(tarpath)

!python /content/stylegan2-ada-pytorch/dataset_tool.py --source /content/flattendata --dest /content/dataset